<a href="https://colab.research.google.com/github/arshiya803/AIML-2303A51389/blob/main/Lab03_AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 Implement the AI Game Strategy

Part 1 –(a). Install the Python Libraries required for Game Strategy. [CO2]

1. Install the python libraries - collections, random, math, functools,
cache = functools.lru cache(10**6)
2. Implement a Game Class Constructor using action, is terminal, result, utility functions
3. A game is similar to a problem, but it has a terminal test instead of a goal test, and a
utility for each terminal state.
4. Create a game subclass and implement actions, result, is terminal, and utility.
5. You will also need to set the initial attribute to the initial state; this can be done in the
constructor.


In [6]:
class Game:
  def actions(self, state ):
         """Return a collection of the allowable moves from this state ."""
         raise NotImplementedError
  def result(self, state , move ):
        """Return the state that results from making a move from a state ."""
        raise NotImplementedError
  def is_terminal (self, state ):
         """Return True i f this is a final state for the game."""
         return not self.actions(state)
  def utility(self, state , player ):
        """Return the value of this final state to player ."""
        raise NotImplementedError

6. Implement a Player Game using the Game Class Constructor.

In [13]:
def play_game (game , strategies : dict , verbose=False ):
      """Play a turn−taking game. ‘strategies ‘ is a {player name: function} dict ,
      where function(state , game) is used to get the player ’s move."""
      state = game.initial
      while not game.is_terminal (state ):
         player = state.to_move
         move = strategies [player ](game , state)
         state = game.result(state , move)
         if verbose:
           print('Player', player , 'move:', move)
           print(state)
           return state



Part 2 – Implement the Game Strategy Algorithms. [CO3]

1. Implement the MiniMax Search Algorithm

In [19]:
def minimax search (game , state ):
      """Search game tree to determine best move; return (value , move) pair ."""
      player = state. to move
      def max_value (state ):
        if game. is terminal (state ):
        return game.utility(state , player), None
        v, move = −infinity , None
        for a in game.actions(state ):
        v2 , = min value (game.result(state , a))
        if v2 > v:
        v, move = v2 , a
        return v, move
     def min_value (state ):
        if game. is terminal (state ):
        return game.utility(state , player), None
        v, move = +infinity , None
       for a in game.actions(state ):
       v2 , = max value (game.result(state , a))
       if v2 < v:
       v, move = v2 , a
       return v, move
       return max value (state)
       infinity = math.inf

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)